# Step1: Extract the genes that passed the wet lab tests

In [1]:
import os
import glob
import pandas as pd

In [ ]:
df_pass = pd.read_excel('../part1/pass_primers-pst-2024.xlsx', usecols=[1,3,5])
combined_df = pd.concat([pd.read_csv(f) for f in glob.glob('../part1/primer-batches/*.csv')],ignore_index=True)
df_pass = df_pass.merge(combined_df[['PrimerSeq_F', 'PrimerLoc_F', 'PrimerLoc_R','GeneLength_no_padding', 'GeneLength_with_padding','Primer_coverage']],on='PrimerSeq_F',how='left')
df_pass.to_csv('../part1/edited-pass_primers-pst-2024.csv', index=False)

In [3]:
ref_dir = '../REF'
ref_fasta = os.path.join(ref_dir,'new_reference_all_genes','pst104e.fasta')
ref_gff3 = os.path.join(ref_dir,'new_reference_all_genes','pst104e.gff3')
out_fasta = os.path.join(ref_dir,'new_reference_marple_genes','new_pst104e_marple_genes.fasta')
out_gff3 = os.path.join(ref_dir,'new_reference_marple_genes','new_pst104e_marple_genes.gff3')

# Step2: Fix gene naming

In [ ]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import re
import csv
from collections import defaultdict

def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join([complement.get(base, base) for base in reversed(seq)])

def extract_gene_sequences(gff_file, fasta_file, padding=0):
    genome = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        genome[record.id] = str(record.seq)
    
    genes = {}
    with open(gff_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9 or fields[2].lower() != 'gene':
                continue
                
            chrom = fields[0]
            start = int(fields[3])
            end = int(fields[4])
            strand = fields[6]
            
            attributes = fields[8]
            match = re.search(r'ID=([^;]+)', attributes) or re.search(r'gene_id=([^;]+)', attributes)
            if not match:
                continue
            gene_id = match.group(1)
            
            if chrom in genome:
                padded_start = max(1, start - padding)
                padded_end = min(len(genome[chrom]), end + padding)
                sequence = genome[chrom][padded_start-1:padded_end]
                
                genes[gene_id] = {
                    'chrom': chrom,
                    'start': start,
                    'end': end,
                    'strand': strand,
                    'sequence': sequence,
                    'padded_start': padded_start,
                    'padded_end': padded_end,
                    'length_no_padding': end - start + 1,
                    'length_with_padding': padded_end - padded_start + 1
                }
    
    return genes

def find_primer_locations(sequence, primer_f, primer_r):
    """Find primer locations considering multiple possible orientations."""
    # Get reverse complements
    primer_f_rc = reverse_complement(primer_f)
    primer_r_rc = reverse_complement(primer_r)
    
    # Store all possible valid primer arrangements
    valid_arrangements = []
    
    # Case 1: Standard orientation (Forward → Reverse complement)
    f_loc = sequence.find(primer_f)
    r_rc_loc = sequence.find(primer_r_rc)
    
    if f_loc >= 0 and r_rc_loc >= 0 and f_loc < r_rc_loc:
        valid_arrangements.append({
            'f_loc': f_loc + 1,  # 1-based coordinate
            'r_loc': r_rc_loc + len(primer_r_rc),
            'amplicon_size': r_rc_loc - f_loc + len(primer_r_rc)
        })
    
    # Case 2: Reversed primer pair (Reverse → Forward complement)
    r_loc = sequence.find(primer_r)
    f_rc_loc = sequence.find(primer_f_rc)
    
    if r_loc >= 0 and f_rc_loc >= 0 and r_loc < f_rc_loc:
        valid_arrangements.append({
            'f_loc': r_loc + 1,  # Reverse primer now acts as forward
            'r_loc': f_rc_loc + len(primer_f_rc),
            'amplicon_size': f_rc_loc - r_loc + len(primer_f_rc)
        })
    
    # Case 3: Circular arrangement (Forward at end, Reverse complement at beginning)
    if f_loc >= 0 and r_rc_loc >= 0 and f_loc > r_rc_loc:
        valid_arrangements.append({
            'f_loc': f_loc + 1,
            'r_loc': r_rc_loc + len(primer_r_rc),
            'amplicon_size': len(sequence) - f_loc + r_rc_loc + len(primer_r_rc)  # Wraparound size
        })
    
    # Case 4: Reversed circular (Reverse at end, Forward complement at beginning)
    if r_loc >= 0 and f_rc_loc >= 0 and r_loc > f_rc_loc:
        valid_arrangements.append({
            'f_loc': r_loc + 1,
            'r_loc': f_rc_loc + len(primer_f_rc),
            'amplicon_size': len(sequence) - r_loc + f_rc_loc + len(primer_f_rc)  # Wraparound size
        })
    
    # If any valid arrangement found, return the one with smallest amplicon size
    if valid_arrangements:
        best_match = min(valid_arrangements, key=lambda x: x['amplicon_size'])
        return best_match['f_loc'], best_match['r_loc']
    
    return None, None


def update_primers_with_correct_genes(primers_csv, gff_file, fasta_file, output_csv, changes_csv, padding=1000):
    primers_df = pd.read_csv(primers_csv)
    
    print("Extracting gene sequences from reference...")
    genes = extract_gene_sequences(gff_file, fasta_file, padding)
    print(f"Extracted {len(genes)} genes from GFF3")
    
    changes = []
    missing_fields_completed = []
    
    # For each primer pair, find the correct gene
    updated_rows = []
    for _, row in primers_df.iterrows():
        original_gene_id = row['GeneID']
        primer_f = row['PrimerSeq_F']
        primer_r = row['PrimerSeq_R']
        
        best_match = None
        best_f_loc = None
        best_r_loc = None
        
        # Check if the current gene ID exists in our genes dictionary
        if original_gene_id in genes:
            # Check if the primers match this gene
            sequence = genes[original_gene_id]['sequence']
            f_loc, r_loc = find_primer_locations(sequence, primer_f, primer_r)
            
            if f_loc is not None and r_loc is not None:
                best_match = original_gene_id
                best_f_loc = f_loc
                best_r_loc = r_loc
                best_gene_info = genes[original_gene_id]
        
        # If no match found with the original gene ID, search all genes
        if best_match is None:
            for gene_id, gene_info in genes.items():
                sequence = gene_info['sequence']
                f_loc, r_loc = find_primer_locations(sequence, primer_f, primer_r)

                if f_loc is not None and r_loc is not None:
                    # Found a match
                    best_match = gene_id
                    best_f_loc = f_loc
                    best_r_loc = r_loc
                    best_gene_info = gene_info
                    break  # Take the first match
        
        new_row = row.copy()
        
        # Case 1: Found a different gene match
        if best_match and best_match != original_gene_id:
            changes.append((original_gene_id, best_match))
            
            # Update gene ID and locations
            new_row['GeneID'] = best_match
            new_row['PrimerLoc_F'] = best_f_loc
            new_row['PrimerLoc_R'] = best_r_loc
            new_row['GeneLength_no_padding'] = best_gene_info['length_no_padding']
            new_row['GeneLength_with_padding'] = best_gene_info['length_with_padding']
            new_row['Primer_coverage'] = best_r_loc - best_f_loc + 1
        
        # Case 2: Same gene match but need to complete missing fields
        elif best_match and best_match == original_gene_id:
            # Check for missing or zero values in important fields
            fields_updated = False
            
            # List of fields to check and update if missing
            fields_to_check = [
                ('PrimerLoc_F', best_f_loc),
                ('PrimerLoc_R', best_r_loc),
                ('GeneLength_no_padding', best_gene_info['length_no_padding']),
                ('GeneLength_with_padding', best_gene_info['length_with_padding'])
            ]
            
            for field_name, new_value in fields_to_check:
                # Check if field is missing, empty, or zero
                # if field_name not in new_row or pd.isna(new_row[field_name]) or new_row[field_name] == 0:
                new_row[field_name] = new_value
                fields_updated = True
            
            # Calculate primer coverage if missing
            if 'Primer_coverage' not in new_row or pd.isna(new_row['Primer_coverage']) or new_row['Primer_coverage'] == 0:
                new_row['Primer_coverage'] = best_r_loc - best_f_loc + 1
                fields_updated = True
                
            if fields_updated:
                missing_fields_completed.append(original_gene_id)
                
        updated_rows.append(new_row)
    
    # Create DataFrame from updated rows
    updated_df = pd.DataFrame(updated_rows)
    
    # Drop rows with empty important fields
    original_count = len(updated_df)
    important_fields = ['GeneID', 'PrimerSeq_F', 'PrimerSeq_R', 'PrimerLoc_F', 'PrimerLoc_R']
    updated_df = updated_df.dropna(subset=important_fields)
    rows_dropped = original_count - len(updated_df)
    
    if rows_dropped > 0:
        print(f"Dropped {rows_dropped} rows with empty important fields")
    
    # Save updated primers
    updated_df.to_csv(output_csv, index=False)
    print(f"Updated primers saved to {output_csv}")
    
    # Save changes
    if changes:
        with open(changes_csv, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Original_GeneID', 'New_GeneID'])
            writer.writerows(changes)
        print(f"Gene ID changes saved to {changes_csv}")
        print(f"Total gene ID changes: {len(changes)}")
    
    # Report on completed fields
    if missing_fields_completed:
        print(f"Completed missing fields for {len(missing_fields_completed)} genes")
        with open('../part1/completed_fields.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['GeneID'])
            for gene_id in missing_fields_completed:
                writer.writerow([gene_id])
        print(f"List of genes with completed fields saved to completed_fields.csv")

if __name__ == "__main__":
    primers_csv = "../part1/edited-pass_primers-pst-2024.csv"
    gff_file = ref_gff3
    fasta_file = ref_fasta
    output_csv = "../part1/updated_primers_new-code.csv"
    changes_csv = "../part1/gene_id_changes.csv"
    
    update_primers_with_correct_genes(primers_csv, gff_file, fasta_file, output_csv, changes_csv)

Extracting gene sequences from reference...
Extracted 30249 genes from GFF3
Dropped 29 rows with empty important fields
Updated primers saved to ../updated_primers_new-code.csv
Gene ID changes saved to ../gene_id_changes.csv
Total gene ID changes: 26
Completed missing fields for 294 genes
List of genes with completed fields saved to completed_fields.csv


# Step3: Create a new reference assembly for PST104E, based on those genes

In [8]:
import os
import gffutils
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [9]:
def extract_genes(gff3_path, fasta_path, gene_ids, output_fasta):
    db_path = os.path.splitext(gff3_path)[0] + '.db'
    if not os.path.exists(db_path):
        gffutils.create_db(gff3_path, db_path, merge_strategy='create_unique')
    
    db = gffutils.FeatureDB(db_path)
    scaffold_records = SeqIO.to_dict(SeqIO.parse(fasta_path, "fasta"))
    
    extracted_records = []
    for gene_id in gene_ids:
        try:
            gene = db[gene_id]
        except KeyError:
            print(f"Warning: Gene {gene_id} not found in GFF3")
            continue
        
        scaffold_id = gene.seqid
        if scaffold_id not in scaffold_records:
            print(f"Warning: Scaffold {scaffold_id} not found in FASTA")
            continue
        
        scaffold_seq = scaffold_records[scaffold_id].seq
        start = max(0, gene.start - 1)  # 0-based
        end = gene.end
        gene_seq = scaffold_seq[start:end]
        
        new_record = SeqIO.SeqRecord(
            gene_seq,
            id=gene_id,
            # description=f"+/- {padding} kb"
        )
        extracted_records.append(new_record)
        # print(f"Length of {gene_id}: {len(gene_seq)}")
        
    SeqIO.write(extracted_records, output_fasta, "fasta")
    return extracted_records

In [10]:

def adapt_gff3(gff3_path, gene_ids, output_gff3):
    db_path = os.path.splitext(gff3_path)[0] + '.db'
    if not os.path.exists(db_path):
        gffutils.create_db(gff3_path, db_path, merge_strategy='create_unique')
    
    db = gffutils.FeatureDB(db_path)
    adapted_features = []
    
    for gene_id in gene_ids:
        try:
            gene = db[gene_id]
        except KeyError:
            print(f"Warning: Gene {gene_id} not found in GFF3")
            continue
        
        original_start = gene.start
        original_end = gene.end
        gene_length = original_end - original_start + 1
        
        # gene.start = 1001
        # gene.end = 1000 + gene_length
        gene.start = 1
        gene.end = gene_length
        gene.seqid = gene_id
        adapted_features.append(gene)
        
        def get_all_children(parent_id):
            children = []
            for child in db.children(parent_id):
                children.append(child)
                children.extend(get_all_children(child.id))
            return children
        
        for child in get_all_children(gene_id):
            if child.id == gene_id:
                continue
            
            # child.start = child.start - original_start + 1001
            # child.end = child.end - original_start + 1001
            child.start = child.start - original_start + 1
            child.end = child.end - original_start + 1
            child.seqid = gene_id
            adapted_features.append(child)
    
    with open(output_gff3, 'w') as f:
        for feature in adapted_features:
            f.write(str(feature) + '\n')

In [ ]:
new_list_of_genes = pd.read_csv('../part1/updated_primers_new-code.csv')['GeneID'].unique()
extract_genes(ref_gff3,ref_fasta, new_list_of_genes, out_fasta)
adapt_gff3(ref_gff3, new_list_of_genes, out_gff3)